In [88]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [89]:
import pandas as pd


# Lineitem

l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX",
                "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT", "L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()

l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../data_storage/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

# Part

p_columnnames = ["P_PARTKEY", "P_NAME", "P_MFGR", "P_BRAND", "P_TYPE", "P_SIZE", "P_CONTAINER", "P_RETAILPRICE", "P_COMMENT"]

for i in range(len(p_columnnames)):
    p_columnnames[i] = p_columnnames[i].lower()
    
p_data_types = {
    'p_partkey': int, 
    'p_name': str,
    'p_mfgr': str,
    'p_brand': str,
    'p_type': str,
    'p_size': int,
    'p_container': str,
    'p_retailprice': float,
    'p_comment': str
}

p_parse_dates = []

# Don't set indexes, as we can't access them with Pandas selection!
part = pd.read_table("../data_storage/part.tbl.csv", sep="|", names=p_columnnames, dtype=p_data_types, parse_dates=p_parse_dates)


In [90]:
df_scan_1 = part[(part.p_brand == 'Brand#23') & (part.p_container == 'MED BOX')]
df_scan_1 = df_scan_1[['p_partkey', 'p_brand', 'p_container']]
df_scan_2 = lineitem[['l_partkey', 'l_quantity']]
df_group_1 = df_scan_2 \
    .groupby(['l_partkey'], sort=False, as_index=False) \
    .agg(
        l_quantitymean=('l_quantity', 'mean'),
    )
print(f"df_group_1: {len(df_group_1)}")
df_group_1['l_quantitymean1'] = df_group_1.l_quantitymean * 0.2
df_join_1 = df_scan_1.merge(df_group_1, left_on=['p_partkey'], right_on=['l_partkey'], how='inner', sort=False)
print(f"df_join_1: {len(df_join_1)}")
df_scan_3 = lineitem[['l_partkey', 'l_quantity', 'l_extendedprice', 'l_linenumber', 'l_orderkey']]
df_join_2 = df_join_1.merge(df_scan_3, left_on=['l_partkey', 'p_partkey'], right_on=['l_partkey', 'l_partkey'], how='inner', sort=False)
df_filter_1 = df_join_2[(df_join_2.l_quantity < df_join_2.l_quantitymean1)]
print(f"df_filter_1: {len(df_filter_1)}")
df_aggr_1 = pd.DataFrame()
df_aggr_1['l_extendedpricesum'] = [df_filter_1.l_extendedprice.sum()]
df_aggr_1['l_extendedpricesum1'] = (df_aggr_1.l_extendedpricesum / 7.0)
df_output_1 = df_aggr_1.rename(columns={'l_extendedpricesum1': 'avg_yearly'})
df_output_1 = df_output_1[['avg_yearly']]

df_group_1: 200000
df_join_1: 204
df_filter_1: 587


In [91]:
df_output_1

,avg_yearly
0,348406.0542857143


In [100]:
import json
json_data = None
with open("../../SDQL/aggrs/test.json") as f:
    json_data = json.load(f)

In [93]:
print(f"SDQLpy Size: {len(json_data['Result']['l_partkey_x'])}")

SDQLpy Size: 200000


In [98]:
known_partkeys = list(df_group_1['l_partkey'])

In [96]:
import random

def do_average(data: list) -> float:
    return sum(data) / len(data)

choices = 5
counter = 0
while counter < choices:
    target_l_partkey = random.choice(known_partkeys)
    print(f"Using l_partket: {target_l_partkey}")
    pandas_mean = list(df_group_1[df_group_1['l_partkey'] == target_l_partkey]['l_quantitymean'])[0]
    print(f"The Pandas Mean was: {pandas_mean}")
    sdqlpy_mean = json_data['Result']['avg_l_quantity'][[int(x) for x in json_data['Result']['l_partkey_x']].index(target_l_partkey)]
    print(f"The SDQLpy Mean was: {sdqlpy_mean}")
    lineitem_real_mean = list(lineitem[lineitem['l_partkey'] == target_l_partkey]['l_quantity'])
    print(f"The Lineitem Real Mean was: {do_average(lineitem_real_mean)}")
    print("---" * 5)
    counter += 1

Using l_partket: 176898
The Pandas Mean was: 26.41176470588235
The SDQLpy Mean was: 56.12
The Lineitem Real Mean was: 26.41176470588235
---------------
Using l_partket: 121083
The Pandas Mean was: 22.4
The SDQLpy Mean was: 39.53
The Lineitem Real Mean was: 22.4
---------------
Using l_partket: 125865
The Pandas Mean was: 25.448275862068964
The SDQLpy Mean was: 67.09
The Lineitem Real Mean was: 25.448275862068964
---------------
Using l_partket: 87917
The Pandas Mean was: 24.875
The SDQLpy Mean was: 41.89
The Lineitem Real Mean was: 24.875
---------------
Using l_partket: 197804
The Pandas Mean was: 24.548387096774192
The SDQLpy Mean was: 42.28
The Lineitem Real Mean was: 24.548387096774192
---------------


In [101]:
used_l_partkeys = [176898, 121083, 125865, 87917, 197804]
for target_l_partkey in used_l_partkeys: 
    # target_l_partkey = random.choice(known_partkeys)
    print(f"Using l_partket: {target_l_partkey}")
    pandas_mean = list(df_group_1[df_group_1['l_partkey'] == target_l_partkey]['l_quantitymean'])[0]
    print(f"The Pandas Mean was: {pandas_mean}")
    sdqlpy_mean = json_data['Result']['avg_l_quantity'][[int(x) for x in json_data['Result']['l_partkey_x']].index(target_l_partkey)]
    print(f"The SDQLpy Mean was: {sdqlpy_mean}")
    lineitem_real_mean = list(lineitem[lineitem['l_partkey'] == target_l_partkey]['l_quantity'])
    print(f"The Lineitem Real Mean was: {do_average(lineitem_real_mean)}")
    print("---" * 5)

Using l_partket: 176898
The Pandas Mean was: 26.41176470588235
The SDQLpy Mean was: 56.12
The Lineitem Real Mean was: 26.41176470588235
---------------
Using l_partket: 121083
The Pandas Mean was: 22.4
The SDQLpy Mean was: 37.33
The Lineitem Real Mean was: 22.4
---------------
Using l_partket: 125865
The Pandas Mean was: 25.448275862068964
The SDQLpy Mean was: 67.09
The Lineitem Real Mean was: 25.448275862068964
---------------
Using l_partket: 87917
The Pandas Mean was: 24.875
The SDQLpy Mean was: 44.22
The Lineitem Real Mean was: 24.875
---------------
Using l_partket: 197804
The Pandas Mean was: 24.548387096774192
The SDQLpy Mean was: 40.05
The Lineitem Real Mean was: 24.548387096774192
---------------
